### FE630 - Homework #2

**Author**: Sid Bhatia

**Date**: May 7th, 2023

**Pledge**: I pledge my honor that I have abided by the Stevens Honor System.

**Professor**: Papa Momar Ndiaye

#### Topics

- Algebra & Optimization;
- Geometry of Efficient Frontiers;
- Applications of One-Fund & Two-Fund Theorems.

#### P1 - Optimization w/Equality Constraints (40 pts)

Consider the optimization problem **Max Expected Return w/Target Risk**:

$$
\begin{cases}
\underset{\omega_1, \omega_2}{\max} & R_P (\omega_1, \omega_2) = \mu_1 \omega_1 + \mu_2 \omega_2 \tag{1} \\
\text{s.t.} & \sqrt{\sigma_1^2 \omega_1^2 + 2 \rho_{1, 2} \sigma_1 \sigma_2 \omega_1 \omega_2 + \sigma_2^2 \omega_2^2} = \sigma_T \\
& \omega_1 + \omega_2 = 1
\end{cases}
$$

where we have two securities with **Expected Returns** $ \mu_1 $ and $ \mu_2 $ for the column vector $ (\mu_1, \mu_2)^\intercal \in \mathbb{R}^{2 \times 1} $, **volatilities** $ (\sigma_1, \sigma_2) \in \mathbb{R}^{+} $, and **Pearson correlation coefficient** $ \rho_{1, 2} \in [-1, 1] $. Additionally, $ \sigma_T \in \mathbb{R}^+ $ denotes the **target risk/vol**.